<a href="https://colab.research.google.com/github/vagabundoo/predicting_financial_markets_deep_learning/blob/master/googleCollabNbs/newWorkflowTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd
import numpy as np
import os
import keras
!pip install ta
import ta

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
aegn = pd.read_csv('gdrive/My Drive/financial-predictions/inputData/aegn.us.txt', 
                   parse_dates=True)

In [4]:
display(aegn.head())
display(aegn.tail())

,Date,Open,High,Low,Close,Volume,OpenInt
0,2011-10-27,14.00,16.34,14.00,15.97,489046,0
1,2011-10-28,15.00,15.98,15.00,15.57,399919,0
2,2011-10-31,15.34,16.08,14.78,14.79,321200,0
3,2011-11-01,14.85,14.92,14.18,14.35,317000,0
4,2011-11-02,15.00,15.68,15.00,15.54,522781,0


,Date,Open,High,Low,Close,Volume,OpenInt
1516,2017-11-06,26.06,27.180,25.8500,26.97,302163,0
1517,2017-11-07,26.95,27.185,25.6801,26.27,403116,0
1518,2017-11-08,26.06,27.190,25.9500,27.11,434933,0
1519,2017-11-09,27.06,27.480,26.5500,27.19,330624,0
1520,2017-11-10,27.05,27.290,26.7300,26.89,208426,0


In [0]:
lenAegn = len(aegn)
end_train = round(0.7 * lenAegn)
start_valid = end_train + 1
end_valid = round(0.85 * lenAegn)
start_test = end_valid + 1

In [6]:
aegn['return'] = aegn['Close'] / aegn['Close'].shift() - 1 
aegn.head()

,Date,Open,High,Low,Close,Volume,OpenInt,return
0,2011-10-27,14.00,16.34,14.00,15.97,489046,0,NaN
1,2011-10-28,15.00,15.98,15.00,15.57,399919,0,-0.025047
2,2011-10-31,15.34,16.08,14.78,14.79,321200,0,-0.050096
3,2011-11-01,14.85,14.92,14.18,14.35,317000,0,-0.029750
4,2011-11-02,15.00,15.68,15.00,15.54,522781,0,0.082927


In [0]:
aegn['label'] = np.where(aegn['return'] > 0, 1, 0)

In [0]:
aegn["std_return"] = (aegn["return"] - aegn["return"][:start_valid].mean()) / aegn["return"][:start_valid].std()
aegn["std_volume"] = (aegn["Volume"] - aegn["Volume"].rolling(50).mean()) / aegn["Volume"].rolling(50).std()

In [9]:
aegn.head()

,Date,Open,High,Low,Close,Volume,OpenInt,return,label,std_return,std_volume
0,2011-10-27,14.00,16.34,14.00,15.97,489046,0,NaN,0,NaN,NaN
1,2011-10-28,15.00,15.98,15.00,15.57,399919,0,-0.025047,0,-1.163213,NaN
2,2011-10-31,15.34,16.08,14.78,14.79,321200,0,-0.050096,0,-2.312454,NaN
3,2011-11-01,14.85,14.92,14.18,14.35,317000,0,-0.029750,0,-1.378976,NaN
4,2011-11-02,15.00,15.68,15.00,15.54,522781,0,0.082927,1,3.790520,NaN


In [0]:
aegn.dropna(inplace=True)

In [0]:
train_generator = TimeseriesGenerator(aegn[["std_return", "std_volume"]].values, aegn[["label"]].values,
                                      length=30, batch_size=64, end_index=start_valid-1)
val_generator = TimeseriesGenerator(aegn[["std_return", "std_volume"]].values, aegn[["label"]].values,
                                    length=30, batch_size=64, start_index=start_valid,
                                    end_index=start_test-1)
test_generator = TimeseriesGenerator(aegn[["std_return", "std_volume"]].values, aegn[["label"]].values,
                                     length=30, batch_size=64, start_index=start_test)

In [0]:
def model_fn(params):
    model = keras.Sequential()
    model.add(keras.layers.CuDNNLSTM(params["lstm_size"], input_shape=(30, 2)))
    model.add(keras.layers.Dropout(params["dropout"]))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer=keras.optimizers.Adam(params["learning_rate"]),
                  loss="binary_crossentropy", metrics=["accuracy"])

    callbacks = [keras.callbacks.EarlyStopping(monitor="val_acc", patience=5,
                                                  restore_best_weights=True)]
    history = model.fit_generator(train_generator, validation_data=val_generator,
                                  callbacks=callbacks, epochs=100, verbose=0).history
    return (history, model)

In [0]:
def random_search(model_fn, search_space, n_iter, search_dir):
    results = []
    os.mkdir(search_dir)
    best_model_path = os.path.join(search_dir, "best_model.h5")
    results_path = os.path.join(search_dir, "results.csv")
    for i in range(n_iter):
        params = {k: v[np.random.randint(len(v))] for k, v in search_space.items()}
        history, model = model_fn(params)
        epochs = np.argmax(history["val_acc"]) + 1
        result = {k: v[epochs - 1] for k, v in history.items()}
        params["epochs"] = epochs
        if i == 0:
            best_val_acc = result["val_acc"]
            model.save(best_model_path)
        if result["val_acc"] > best_val_acc:
            best_val_acc = result["val_acc"]
            model.save(best_model_path)
        result = {**params, **result}
        results.append(result)
        keras.backend.clear_session()
        print(f"iteration {i + 1} – {', '.join(f'{k}:{v:.4g}' for k, v in result.items())}")
    best_model = keras.models.load_model(best_model_path)
    results = pd.DataFrame(results)
    results.to_csv(results_path)
    return (results, best_model)

In [0]:
search_space = {"lstm_size": np.linspace(50, 200, 16, dtype=int),
                "dropout": np.linspace(0, 0.4, 9),
                "learning_rate": np.linspace(0.004, 0.01, 13)}

In [22]:
results, best_model = random_search(model_fn, search_space, 200, "search")

FileExistsError: ignored

In [0]:
model_fn()  